Splitting the data into chuncks


In [ ]:
import pandas as pd
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Load CSV file
df = pd.read_csv("synthetic_knowledge_items.csv")

# 2. Drop rows with missing or empty 'ki_text'
df = df[df['ki_text'].notna() & df['ki_text'].str.strip().ne('')]

# 3. Text splitter to split each ki_text into ~2-3 chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       # Set chunk size
    chunk_overlap=300     # Allow small overlap for context
)

# 4. Create list of chunks
chunked_docs = []

for idx, row in df.iterrows():
    text = row["ki_text"].strip()
    # Split into chunks
    chunks = splitter.create_documents([text])
    # Add row number
    for chunk in chunks:
        chunk.metadata["row"] = idx
    chunked_docs.extend(chunks)

# 5. Print a few examples
print(f" Total chunks created: {len(chunked_docs)}")
for i, doc in enumerate(chunked_docs[:3], 1):
    print(f"Chunk {i}:\n{doc.page_content}\n{'-'*50}")


 Total chunks created: 383
Chunk 1:
**Setting Up a Mobile Device for Company Email**

**Prerequisites:**

* Mobile device with a supported operating system (iOS, Android, or Windows)
* Company email account credentials
* Mobile device management (MDM) profile installed (if required by company policy)

**Step 1: Ensure Mobile Device Management (MDM) Profile is Installed (if required)**

If your company requires MDM for mobile devices, ensure that the profile is installed on your device. This profile will allow your device to connect to the company network and access company email. If you are unsure whether MDM is required, contact your IT department for assistance.

**Step 2: Set Up Email Account on Mobile Device**
--------------------------------------------------
Chunk 2:
**Step 2: Set Up Email Account on Mobile Device**

1. Go to the Settings app on your mobile device.
2. Select "Mail" or "Email" (depending on your device's operating system).
3. Tap "Add Account" or "Create a new acc

Converting the Chuncks into embeddings and storing them with FAISS

In [ ]:
!pip install -U langchain-community
!pip install faiss-cpu
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

emb_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #model for vectorising
vectorstore = FAISS.from_documents(chunked_docs, emb_model) #Vectors stored here

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.3 MB/s eta 0:00:00


<ipython-input-2-3e7a56f1697d>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') #model for vectorising
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using BM25 to retrive

In [ ]:
!pip install rank_bm25
from langchain.retrievers import BM25Retriever

# query = "how to configure Email on Android Device"
query = "how to reset a jammed printer"
bm25_retriever = BM25Retriever.from_documents(chunked_docs)
bm25_retriever.k = 3  # Number of top documents to return

bm25_results = bm25_retriever.get_relevant_documents(query)

for i, doc in enumerate(bm25_results, 1):
    print(f"\n BM25 Result {i}:\n{doc.page_content}\n{'-'*50}")




 BM25 Result 1:
**Step 8: Power On and Test**

Turn the printer back on and test it by printing a test page or a document. If the printer is still experiencing issues, refer to the manufacturer's troubleshooting guide or contact the IT helpdesk for further assistance.

**Additional Tips:**

* Regularly clean and maintain your printer to prevent paper jams and other issues.
* Use high-quality paper that is designed for printing to reduce the likelihood of jams.
* Avoid overloading the paper tray, as this can cause paper jams and other issues.
* If you are unsure about how to reset a jammed printer or if the issue persists after following these steps, contact the IT helpdesk for assistance.
--------------------------------------------------

 BM25 Result 2:
**Step 1: Gather Necessary Information**

Before setting up a new printer on the network, gather the following information:

* The printer's model number and manufacturer
* The printer's IP address (if it has a static IP address)
* T

Using FAISS to retrive

In [ ]:
faiss_results = vectorstore.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(faiss_results, 1):
    print(f"FAISS Result {i} \n{doc.page_content}\n{'-'*50}")

# normalize FAISS similarity scores
faiss_scored = []
for doc, score in faiss_results:
    normalized_score = score / max([s for c, s in faiss_results])  # scale to 0–1
    faiss_scored.append((doc, normalized_score))


FAISS Result 1 
**Resetting a Jammed Printer**

**Step 1: Turn Off the Printer**

Immediately turn off the printer to prevent any further damage or paper jams. Locate the power button, usually found on the top or front of the printer, and press it to shut down the device.

**Step 2: Open the Printer**

Locate the printer's access panel, which varies depending on the printer model. Common locations include the top, front, or back of the printer. Open the panel by pressing the release latch or sliding it open. This will give you access to the internal mechanisms.

**Step 3: Identify and Remove Jammed Paper**

Carefully inspect the printer's internal mechanisms to locate the jammed paper. Gently pull out any visible paper scraps or torn pieces. Be cautious not to touch any internal components or rollers, as oils from your skin can cause damage.

**Step 4: Check for Obstructions**
--------------------------------------------------
FAISS Result 2 
**Step 8: Power On and Test**

Turn the pri

Re-ranking Strategy

In [ ]:
doc_scores = {} # Stores the combined score for each result
# Adding FAISS scores
for doc, score in faiss_scored:
    doc_text = doc.page_content
    if doc_text not in doc_scores:
        doc_scores[doc_text] = {"bm25": 0.0, "faiss": 0.0}
    doc_scores[doc_text]["faiss"] = score

# Adding BM25 scores
for rank, doc in enumerate(bm25_results):
    doc_text = doc.page_content
    if doc_text not in doc_scores:
        doc_scores[doc_text] = {"bm25": 0.0, "faiss": 0.0}
    score = 1.0 - (rank / len(bm25_results))
    doc_scores[doc_text]["bm25"] = score

combined_results = [] # Final results

for content, scores in doc_scores.items():
    combined_score = (scores["bm25"] + scores["faiss"]) / 2  # average
    combined_results.append((content, combined_score))

# Sort by combined score
combined_results.sort(key=lambda x: x[1], reverse=True)


In [ ]:
print("Re-ranked Top Results:\n")
context = ""
for i, (text, score) in enumerate(combined_results[:3], 1): # Showing top 3 results
    context += f"Result {i} (score={score:.2f}):\n{text}\n{'-'*50}\n"
print(context)

Re-ranked Top Results:

Result 1 (score=1.00):
**Step 8: Power On and Test**

Turn the printer back on and test it by printing a test page or a document. If the printer is still experiencing issues, refer to the manufacturer's troubleshooting guide or contact the IT helpdesk for further assistance.

**Additional Tips:**

* Regularly clean and maintain your printer to prevent paper jams and other issues.
* Use high-quality paper that is designed for printing to reduce the likelihood of jams.
* Avoid overloading the paper tray, as this can cause paper jams and other issues.
* If you are unsure about how to reset a jammed printer or if the issue persists after following these steps, contact the IT helpdesk for assistance.
--------------------------------------------------
Result 2 (score=0.50):
Carefully inspect the printer's internal mechanisms to locate the jammed paper. Gently pull out any visible paper scraps or torn pieces. Be cautious not to touch any internal components or rollers,

In [ ]:
# import torch
# print(torch.cuda.get_device_name(0))

Flan-t5-base(didn't work)

In [ ]:
# # pip install accelerate
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")

# prompt = f"""You are a helpful assistant answering based on company documents.

# Retrieved Context:
# {context}

# Question:
#  {query}

# Answer:"""

# input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.to("cuda")

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))


LLaMA with Groq api

In [ ]:
import requests
from google.colab import userdata

api_key = userdata.get('GROQ_API_KEY')


# Prompt sent to LLaMA model
prompt = f"""You are a helpful assistant that answers questions using only the context provided.

Context:
{context}


Question: {query}
Answer:"""

# Groq API endpoint for LLaMA
url = "https://api.groq.com/openai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

data = {
    "model": "llama-3.3-70b-versatile",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    "temperature": 0.9
}

response = requests.post(url, headers=headers, json=data)
answer = response.json()["choices"][0]["message"]["content"]

print("Answer:", answer)


Answer: To reset a jammed printer, follow these steps: 

1. Turn Off the Printer: Immediately turn off the printer to prevent any further damage or paper jams. 
2. Open the Printer: Locate the printer's access panel and open it to access the internal mechanisms.
3. Identify and Remove Jammed Paper: Carefully inspect the internal mechanisms, locate the jammed paper, and gently pull out any visible paper scraps or torn pieces.
4. Check for Obstructions: Inspect the printer's paper path for any obstructions or debris and remove them.
5. Realign the Paper Tray: If the paper tray was dislodged, realign it according to the manufacturer's instructions.
6. Check and Clean the Print Head: Check the print head for any blockages or dried ink and gently clean it with a lint-free cloth or cotton swab.
7. Reassemble the Printer: Put the printer back together.
8. Power On and Test: Turn the printer back on and test it by printing a test page or a document. If issues persist, refer to the manufacturer